## Cell Comparisons

### Introduction

Individual data points within Iris cubes are actually cells, which can be made up of *both* a point and a set of bounds. Cells do not have to be bounded, however in this example we are interested in cells that are.

In a bounded cell, the point defines the precise position at which the point's value holds, while the bounds define the extent over which the point's value is *also true*.

To conceptualise this, imagine that our cell contains one temperature field from a UKV model run. Our cell's point tells us that the value of this given cell is 300K, but our cell is also bounded. This means that our cell's value also holds across the volume described by the bounds. If the UKV model has a resolution of 1.5km and we assume this is true in x, y and z, then we state that our value of 300K is true for a cube 1.5km on a side centred on our cell's point.

This is, of course, best shown with a picture:

![A graphical demonstration of an Iris cell.](./img/cell.png)

We can add to the complexity by now imagining that this model runs in 3-hourly timesteps. Now our cell has gained a time bound too, so our cell becomes a 4D hypercube with our point centred in the middle of our four dimensions' worth of bounds.

### The Problem

To demonstrate the problem, we will choose a much simpler example than our UKV model cell from above.

Our example will be a 1D coordinate with point values ranging from 0 to 10 inclusive, with each point having bounds of +/- 2 from the point's value.

This is displayed graphically in the following image. Cell points are the dots and the 1D bounds are illustrated by the whiskers from each point. The scalar comparison value is shown by the dashed grey line:

![Graphical presentation of the 1D coordinate](./img/1d_coord.png)

Let's set this up using Python and Iris. We will start by importing Iris and checking its version.

In [1]:
import iris
import iris.coords
import iris.cube
import numpy as np
iris.__version__

'3.10.0'

Now we will set up a 1D coordinate, with bounds, as above:

In [2]:
points = np.arange(11)
bounds = np.array(([(x-2, x+2) for x in points]))
example_coord = iris.coords.AuxCoord(points, bounds=bounds, long_name='example')
print (example_coord)

AuxCoord :  example / (unknown)
    points: [ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10]
    bounds: [
        [-2,  2],
        [-1,  3],
        ...,
        [ 7, 11],
        [ 8, 12]]
    shape: (11,)  bounds(11, 2)
    dtype: int64
    long_name: 'example'


Now that we have a coordinate, we can start investigating some cell comparisons. We will compare each cell within our coordinate to the scalar value of 5.

In [3]:
value = 5
for cell in example_coord.cells():
    print(f"{cell} == {value} is {cell==value}")

Cell(point=0, bound=(-2, 2)) == 5 is False
Cell(point=1, bound=(-1, 3)) == 5 is False
Cell(point=2, bound=(0, 4)) == 5 is False
Cell(point=3, bound=(1, 5)) == 5 is True
Cell(point=4, bound=(2, 6)) == 5 is True
Cell(point=5, bound=(3, 7)) == 5 is True
Cell(point=6, bound=(4, 8)) == 5 is True
Cell(point=7, bound=(5, 9)) == 5 is True
Cell(point=8, bound=(6, 10)) == 5 is False
Cell(point=9, bound=(7, 11)) == 5 is False
Cell(point=10, bound=(8, 12)) == 5 is False


The behaviour we observe is that whenever the value is *within the range of the bounds* the equality test returns true. Thus the cells with point values 3 &le; p &le; 7 are all classified as equalling our test value of 5.

### The Solution

It is worth noting right at the start that the above is not a *problem* with Iris. It is valid equality testing &mdash; otherwise, we would not be able to test for equality within bounds at all.

Clearly, however, there will be occasions when it is necessary to equality test bounded cells more strictly by testing for equality to the point *only*.

In [4]:
for cell in example_coord.cells():
    print(f"{cell.point} == {value} is {cell.point==value}")

0 == 5 is False
1 == 5 is False
2 == 5 is False
3 == 5 is False
4 == 5 is False
5 == 5 is True
6 == 5 is False
7 == 5 is False
8 == 5 is False
9 == 5 is False
10 == 5 is False


Here, instead of equality testing to the whole cell, which as can be seen above is composed of both a point value and a bounds range, we equality test to the value of the cell's point specifically. Doing so returns what may be considered the more expected or logical result of cell comparison; that is, cell equality to a scalar value *only when* the scalar value equals the value of the cell's point.

### Example: Cell comparison on an Iris cube

Let's briefly look at performing cell comparisons like this on an Iris cube. We will start by constructing an Iris `DimCoord` with our points and bounds from above and add this to a simple dummy cube:

In [5]:
example_coord = iris.coords.DimCoord(points, bounds=bounds, long_name='example')
dummy_cube = iris.cube.Cube(np.random.random(11))
dummy_cube.add_dim_coord(example_coord, 0)

To perform cell comparisons on our cube we can use an Iris constraint, as shown below. Here the first constraint performs cell comparisons on each cell (points and bounds), and the second performs cell comparisons against each cell's points only:

In [6]:
cell_comparison_constraint = iris.Constraint(example=lambda cell: cell == 5)
cell_point_comparison_constraint = iris.Constraint(example=lambda cell: cell.point == 5)

print (dummy_cube.extract(cell_comparison_constraint).coord())
print (dummy_cube.extract(cell_point_comparison_constraint).coord())

DimCoord :  example / (unknown)
    points: [3, 4, 5, 6, 7]
    bounds: [
        [1, 5],
        [2, 6],
        [3, 7],
        [4, 8],
        [5, 9]]
    shape: (5,)  bounds(5, 2)
    dtype: int64
    long_name: 'example'
DimCoord :  example / (unknown)
    points: [5]
    bounds: [[3, 7]]
    shape: (1,)  bounds(1, 2)
    dtype: int64
    long_name: 'example'
